# Function calling with Assistants API

In [72]:
tools_list = [
{"type": "function",
"function": {
        "name": "get_weather",
        "description": "Determine weather in a certain location",
        "parameters": {
            "type": "object",
            "properties": {
            "location": {
                "type": "string",
                "description": "The city and country e.g. Paris (France)"
                }
            },
            "required": [
                "location"
            ]
        }
    }
}
]

In [79]:
assistant = client.beta.assistants.create(
    name="AI Assistant",
    instructions="You are a personal AI Assistant", 
    model="gpt-4-turbo", 
    tools=tools_list
)

# Step 2: Create a Thread
thread = client.beta.threads.create()

# Step 3: Add a Message to a Thread
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="What is the current weather in Milan?"
)

# Step 4: Run the Assistant
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id,
    instructions="Please address the user as Valentina."
)

In [67]:
import json

json_output = run.json()
data = json.loads(json_output)

for key, value in data.items():
    print(f"{key}: {value}")


id: run_zfRTZMsSRaWHY78bnwBO8NjX
assistant_id: asst_iLa8HAhgddX99aKLJJAwto5u
cancelled_at: None
completed_at: None
created_at: 1711719541
expires_at: 1711720141
failed_at: None
file_ids: []
instructions: Please address the user as Valentina.
last_error: None
metadata: {}
model: gpt-4-turbo
object: thread.run
required_action: None
started_at: None
status: queued
thread_id: thread_yVPsV1GetYeAHJSnGN7ah4B0
tools: [{'function': {'name': 'get_weather', 'description': 'Determine weather in a certain location', 'parameters': {'type': 'object', 'properties': {'location': {'type': 'string', 'description': 'The city and country e.g. Paris (France)'}}, 'required': ['location']}}, 'type': 'function'}]
usage: None


In [58]:
assistant.id

'asst_Ff8lcQP3zyRZfWS3vACwNyDR'

In [56]:
def get_weather(location):
    base_url = "https://api.openweathermap.org/data/2.5/weather?"

    complete_url = base_url + "q=" + location + "&appid=" + api_key + "&units=metric" #set your api key here
    
    response = requests.get(complete_url)
    weather_condition = response.json()["weather"][0]["description"]
    temperature = response.json()["main"]["temp"]
    
    return f"""Here is some information about the weather in {location}:\n
        - The weather is: {weather_condition}.\n
        - The temperature is: {temperature} degrees Celsius.\n
    """

In [82]:
while True:
    # Wait for 5 seconds
    time.sleep(5)

    # Retrieve the run status
    run_status = client.beta.threads.runs.retrieve(
        thread_id=thread.id,
        run_id=run.id
    )
    print(run_status.model_dump_json(indent=4))

    # If run is completed, get messages
    if run_status.status == 'completed':
        messages = client.beta.threads.messages.list(
            thread_id=thread.id
        )

        # Loop through messages and print content based on role
        for msg in messages.data:
            role = msg.role
            content = msg.content[0].text.value
            print(f"{role.capitalize()}: {content}")

        break
    elif run_status.status == 'requires_action':
        print("Function Calling")
        required_actions = run_status.required_action.submit_tool_outputs.model_dump()
        print(required_actions)
        tool_outputs = []
        import json
        for action in required_actions["tool_calls"]:
            func_name = action['function']['name']
            arguments = json.loads(action['function']['arguments'])           
            output = globals()[func_name](**arguments)  # Assuming functions are in the global scope
            tool_outputs.append({
                "tool_call_id": action['id'],
                "output": output
            })
            
        print("Submitting outputs back to the Assistant...")
        client.beta.threads.runs.submit_tool_outputs(
            thread_id=thread.id,
            run_id=run.id,
            tool_outputs=tool_outputs
        )
    else:
        print("Waiting for the Assistant to process...")
        time.sleep(5)

{
    "id": "run_Rys3l7zd7uR94pRx64ETPu8s",
    "assistant_id": "asst_QTz55diqssQmJrvmHxCbe9Y6",
    "cancelled_at": null,
    "completed_at": null,
    "created_at": 1711882075,
    "expires_at": 1711882675,
    "failed_at": null,
    "file_ids": [],
    "instructions": "Please address the user as Valentina.",
    "last_error": null,
    "metadata": {},
    "model": "gpt-4-turbo",
    "object": "thread.run",
    "required_action": {
        "submit_tool_outputs": {
            "tool_calls": [
                {
                    "id": "call_KoKKad9YKkHdqhMut2kh0peX",
                    "function": {
                        "arguments": "{\"location\":\"Milan, Italy\"}",
                        "name": "get_weather"
                    },
                    "type": "function"
                }
            ]
        },
        "type": "submit_tool_outputs"
    },
    "started_at": 1711882076,
    "status": "requires_action",
    "thread_id": "thread_TSFeZyzOPZaH9PEmxu65qFpO",
    "to

In [78]:
client.beta.threads.delete(thread_id = thread.id)

ThreadDeleted(id='thread_Bf4k39NALhPRPlbF4tiAs8dF', deleted=True, object='thread.deleted')